In [1]:
import scrapy
from selenium import webdriver
import sys
sys.path.append("../..")
#from case_scrapy.items import case_Items
from selenium.webdriver.support.ui import Select
from scrapy import Selector
import json
import re
import time
import datetime
import requests
import lxml.html as lh
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Scrape Links

In [225]:
chromedriver_path = "/Users/yuchen.zhang/Downloads/Installations/Chromedriver"
driver = webdriver.Chrome(executable_path = chromedriver_path)

In [226]:
case_links = []
i = 1
first_page_html = requests.get("http://securities.stanford.edu/filings.html?page=1").text
next_page_html = requests.get("http://securities.stanford.edu/filings.html?page=" + str(i+1)).text

while first_page_html != next_page_html:
    try:
        driver.get("http://securities.stanford.edu/filings.html?page=" + str(i))
        sel_case = Selector(text=driver.page_source)
        case_links_raw = sel_case.css('tr:nth-child(n)::attr(onclick)').getall()
        case_links.append(case_links_raw)
        
        i += 1
        next_page_html = requests.get("http://securities.stanford.edu/filings.html?page=" + str(i)).text
    except: break

total_pages = i

In [228]:
# Change the links to actual links
case_links_clean = []
for i in range(0, len(case_links)):
    for j in range(0, len(case_links[i])):
        case_links_clean.append("http://securities.stanford.edu/" + case_links[i][j].replace("window.location=","").replace("'",""))

In [56]:
pd.DataFrame(case_links_clean).to_excel("case_links_all.xlsx")

# Scrape a whole table

In [197]:
page = requests.get("http://securities.stanford.edu/filings.html?page=1")
soup = BeautifulSoup(page.content, 'html.parser')

In [185]:
print(soup.title.text)

Securities Class Action Clearinghouse: Filings Database


In [190]:
# Get headers of the table
case_table = soup.find("table", attrs={"class": "table table-bordered table-striped table-hover"})
table_header = soup.find('thead')
header = table_header.find_all('th')
header = [x.text.strip() for x in header]
print(header)

['Filing Name', 'Filing Date', 'District Court', 'Exchange', 'Ticker']


In [142]:
# Get table of the body, saving element row by row
table_body = soup.find('tbody')
rows = table_body.find_all('tr')
table_list = []
for row in rows:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    table_list.append(cols)

In [196]:
# Save the complete table
summary_table = pd.DataFrame(table_list)
summary_table.columns = header

In [198]:
# Make a function 
def get_summary_table(url):
    # Get page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Get headers of the table
    case_table = soup.find("table", attrs={"class": "table table-bordered table-striped table-hover"})
    table_header = soup.find('thead')
    header = table_header.find_all('th')
    header = [x.text.strip() for x in header]
    
    # Get table of the body, saving element row by row
    table_body = soup.find('tbody')
    rows = table_body.find_all('tr')
    table_list = []
    for row in rows:
        cols=row.find_all('td')
        cols=[x.text.strip() for x in cols]
        table_list.append(cols)
        
    # Save the complete table
    summary_table = pd.DataFrame(table_list)
    summary_table.columns = header
    
    return summary_table

In [238]:
table_all = pd.DataFrame()
for i in range(1, total_pages):
    table_all = table_all.append(get_summary_table("http://securities.stanford.edu/filings.html?page=" + str(i)))

In [240]:
pd.DataFrame(table_all).to_excel("summary_table_all.xlsx")

# Scrape each page